In [1]:
%cd C:/Users/MSI/Desktop/Folders/S5/vision-artificielle/projet/mini-projet2/code

C:\Users\MSI\Desktop\Folders\S5\vision-artificielle\projet\mini-projet2\code


In [2]:
import pybullet as p
import time
import pybullet_data

In [3]:
# p.DIRECT for non-graphical version
p.connect(p.DIRECT)

0

In [4]:
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.setPhysicsEngineParameter(numSolverIterations=10)
p.setTimeStep(1. / 120.)
logId = p.startStateLogging(p.STATE_LOGGING_PROFILE_TIMINGS, "visualShapeBench.json")


In [5]:
#useMaximalCoordinates is much faster then the default reduced coordinates (Featherstone)
p.loadURDF("plane100.urdf", useMaximalCoordinates=True)

0

In [6]:
# Set up PyBullet simulation parameters
p.setGravity(0, 0, -5)

In [7]:
# Load your 3D mesh files into PyBullet
maze_visual_shape = p.createVisualShape(shapeType=p.GEOM_MESH, fileName="models/mazeZ.obj", meshScale=[1, 1, 1])
maze_collision_shape = p.createCollisionShape(shapeType=p.GEOM_MESH, fileName="models/mazeZ_collider.obj")
maze_id = p.createMultiBody(baseMass=0,
                                    baseCollisionShapeIndex=maze_collision_shape,
                                    baseVisualShapeIndex=maze_visual_shape,
                                    useMaximalCoordinates=True)

In [8]:
# Load your 3D mesh files into PyBullet
ground_visual_shape = p.createVisualShape(shapeType=p.GEOM_MESH, fileName="models/groundZ.obj", meshScale=[1, 1, 1])
ground_collision_shape = p.createCollisionShape(shapeType=p.GEOM_MESH, fileName="models/groundZ_collider.obj")
ground_id = p.createMultiBody(baseMass=0,
                                    baseCollisionShapeIndex=ground_collision_shape,
                                    baseVisualShapeIndex=ground_visual_shape,
                                    useMaximalCoordinates=True)

In [9]:
agent_visual_shape = p.createVisualShape(shapeType=p.GEOM_MESH, fileName="models/agentZ.obj", 
                         meshScale=[1, 1, 1])
agent_collision_shape = p.createCollisionShape(shapeType=p.GEOM_MESH, fileName="models/agentZ_collider.obj",
                                               meshScale=[1, 1, 1])
agent_id = p.createMultiBody(baseMass=0.1, baseInertialFramePosition=[0, 0, 0], 
                             baseCollisionShapeIndex=agent_collision_shape, baseVisualShapeIndex=agent_visual_shape,
                             useMaximalCoordinates=True)

In [10]:
import glfw
from OpenGL.GL import *
from OpenGL.GL.shaders import compileProgram, compileShader
import pyrr
from TextureLoader import load_texture
from ObjLoader import ObjLoader
from camera import Camera

cam = Camera()
WIDTH, HEIGHT = 1280, 720
lastX, lastY = WIDTH / 2, HEIGHT / 2
first_mouse = True
left, right, forward, backward = False, False, False, False

agent_pos = pyrr.Vector3([0, 1, -10])
agent_speed = 0.015

# the keyboard input callback
def key_input_clb(window, key, scancode, action, mode):
    global left, right, forward, backward
    if key == glfw.KEY_ESCAPE and action == glfw.PRESS:
        glfw.set_window_should_close(window, True)

    if key == glfw.KEY_W and action == glfw.PRESS:
        forward = True
    elif key == glfw.KEY_W and action == glfw.RELEASE:
        forward = False
    if key == glfw.KEY_S and action == glfw.PRESS:
        backward = True
    elif key == glfw.KEY_S and action == glfw.RELEASE:
        backward = False
    if key == glfw.KEY_A and action == glfw.PRESS:
        left = True
    elif key == glfw.KEY_A and action == glfw.RELEASE:
        left = False
    if key == glfw.KEY_D and action == glfw.PRESS:
        right = True
    elif key == glfw.KEY_D and action == glfw.RELEASE:
        right = False
    # if key in [glfw.KEY_W, glfw.KEY_S, glfw.KEY_D, glfw.KEY_A] and action == glfw.RELEASE:
    #     left, right, forward, backward = False, False, False, False


# do the movement, call this function in the main loop
def do_movement():
    if left:
        cam.process_keyboard("LEFT", 0.015)
    if right:
        cam.process_keyboard("RIGHT", 0.015)
    if forward:
        cam.process_keyboard("FORWARD", 0.015)
    if backward:
        cam.process_keyboard("BACKWARD", 0.015)


# the mouse position callback function
def mouse_look_clb(window, xpos, ypos):
    global first_mouse, lastX

    if first_mouse:
        lastX = xpos
        first_mouse = False

    xoffset = xpos - lastX

    lastX = xpos

    cam.process_mouse_movement(xoffset, 0)


vertex_src = """
# version 330

layout(location = 0) in vec3 a_position;
layout(location = 1) in vec2 a_texture;
layout(location = 2) in vec3 a_normal;

uniform mat4 model;
uniform mat4 projection;
uniform mat4 view;

out vec2 v_texture;

void main()
{
    gl_Position = projection * view * model * vec4(a_position, 1.0);
    v_texture = a_texture;
}
"""

fragment_src = """
# version 330

in vec2 v_texture;

out vec4 out_color;

uniform sampler2D s_texture;

void main()
{
    out_color = texture(s_texture, v_texture);
}
"""


# the window resize callback function
def window_resize_clb(window, width, height):
    glViewport(0, 0, width, height)
    projection = pyrr.matrix44.create_perspective_projection_matrix(45, width / height, 0.1, 100)
    glUniformMatrix4fv(proj_loc, 1, GL_FALSE, projection)


# initializing glfw library
if not glfw.init():
    raise Exception("glfw can not be initialized!")

# creating the window
window = glfw.create_window(WIDTH, HEIGHT, "My OpenGL window", None, None)

# check if window was created
if not window:
    glfw.terminate()
    raise Exception("glfw window can not be created!")

# set window's position
glfw.set_window_pos(window, 400, 200)

# set the callback function for window resize
glfw.set_window_size_callback(window, window_resize_clb)
# set the mouse position callback
glfw.set_cursor_pos_callback(window, mouse_look_clb)
# set the keyboard input callback
glfw.set_key_callback(window, key_input_clb)
# capture the mouse cursor
glfw.set_input_mode(window, glfw.CURSOR, glfw.CURSOR_DISABLED)

# make the context current
glfw.make_context_current(window)

# load here the 3d meshes
cube_indices, cube_buffer = ObjLoader.load_model("models/mazeY.obj")
agent_indices, agent_buffer = ObjLoader.load_model("models/agentY.obj")
ground_indices, ground_buffer = ObjLoader.load_model("models/groundY.obj")

shader = compileProgram(compileShader(vertex_src, GL_VERTEX_SHADER), compileShader(fragment_src, GL_FRAGMENT_SHADER))

# VAO and VBO
VAO = glGenVertexArrays(3)
VBO = glGenBuffers(3)

# cube VAO
glBindVertexArray(VAO[0])
# cube Vertex Buffer Object
glBindBuffer(GL_ARRAY_BUFFER, VBO[0])
glBufferData(GL_ARRAY_BUFFER, cube_buffer.nbytes, cube_buffer, GL_STATIC_DRAW)

# cube vertices
glEnableVertexAttribArray(0)
glVertexAttribPointer(0, 3, GL_FLOAT, GL_FALSE, cube_buffer.itemsize * 8, ctypes.c_void_p(0))
# cube textures
glEnableVertexAttribArray(1)
glVertexAttribPointer(1, 2, GL_FLOAT, GL_FALSE, cube_buffer.itemsize * 8, ctypes.c_void_p(12))
# cube normals
glVertexAttribPointer(2, 3, GL_FLOAT, GL_FALSE, cube_buffer.itemsize * 8, ctypes.c_void_p(20))
glEnableVertexAttribArray(2)


# Agent VAO
glBindVertexArray(VAO[1])
# Agent Vertex Buffer Object
glBindBuffer(GL_ARRAY_BUFFER, VBO[1])
glBufferData(GL_ARRAY_BUFFER, agent_buffer.nbytes, agent_buffer, GL_STATIC_DRAW)

# agent vertices
glEnableVertexAttribArray(0)
glVertexAttribPointer(0, 3, GL_FLOAT, GL_FALSE, agent_buffer.itemsize * 8, ctypes.c_void_p(0))
# agent textures
glEnableVertexAttribArray(1)
glVertexAttribPointer(1, 2, GL_FLOAT, GL_FALSE, agent_buffer.itemsize * 8, ctypes.c_void_p(12))
# agent normals
glVertexAttribPointer(2, 3, GL_FLOAT, GL_FALSE, agent_buffer.itemsize * 8, ctypes.c_void_p(20))
glEnableVertexAttribArray(2)


# Ground VAO
glBindVertexArray(VAO[2])
# Ground Vertex Buffer Object
glBindBuffer(GL_ARRAY_BUFFER, VBO[2])
glBufferData(GL_ARRAY_BUFFER, ground_buffer.nbytes, ground_buffer, GL_STATIC_DRAW)

# Ground vertices
glEnableVertexAttribArray(0)
glVertexAttribPointer(0, 3, GL_FLOAT, GL_FALSE, ground_buffer.itemsize * 8, ctypes.c_void_p(0))
# Ground textures
glEnableVertexAttribArray(1)
glVertexAttribPointer(1, 2, GL_FLOAT, GL_FALSE, ground_buffer.itemsize * 8, ctypes.c_void_p(12))
# Ground normals
glVertexAttribPointer(2, 3, GL_FLOAT, GL_FALSE, ground_buffer.itemsize * 8, ctypes.c_void_p(20))
glEnableVertexAttribArray(2)


textures = glGenTextures(3)
load_texture("textures/maze.jpg", textures[0])
load_texture("textures/agent.jpg", textures[1])
load_texture("textures/ground.jpg", textures[2])


glUseProgram(shader)
glClearColor(0, 0.1, 0.1, 1)
glEnable(GL_DEPTH_TEST)
glEnable(GL_BLEND)
glBlendFunc(GL_SRC_ALPHA, GL_ONE_MINUS_SRC_ALPHA)


#***************************************************************************************** 

projection = pyrr.matrix44.create_perspective_projection_matrix(45, WIDTH / HEIGHT, 0.1, 100)


# eye, target, up
#view = pyrr.matrix44.create_look_at(pyrr.Vector3([0, 0, 12]), pyrr.Vector3([0, 0, 0]), pyrr.Vector3([0, 1, 0]))

model_loc = glGetUniformLocation(shader, "model")
proj_loc = glGetUniformLocation(shader, "projection")
view_loc = glGetUniformLocation(shader, "view")

glUniformMatrix4fv(proj_loc, 1, GL_FALSE, projection)
#glUniformMatrix4fv(view_loc, 1, GL_FALSE, view)

# the main application loop
while not glfw.window_should_close(window):
    glfw.poll_events()
    do_movement()

    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    
    # Move the agent based on the keyboard input
    if left:
        p.applyExternalForce(agent_id, -1, forceObj=[agent_speed, 0, 0], posObj=p.getBasePositionAndOrientation(agent_id)[0], flags=p.WORLD_FRAME)
    if right:
        p.applyExternalForce(agent_id, -1, forceObj=[-agent_speed, 0, 0], posObj=p.getBasePositionAndOrientation(agent_id)[0], flags=p.WORLD_FRAME)
    if forward:
        p.applyExternalForce(agent_id, -1, forceObj=[0, 0, agent_speed], posObj=p.getBasePositionAndOrientation(agent_id)[0], flags=p.WORLD_FRAME)
    if backward:
        p.applyExternalForce(agent_id, -1, forceObj=[0, 0, -agent_speed], posObj=p.getBasePositionAndOrientation(agent_id)[0], flags=p.WORLD_FRAME)

    # Step the PyBullet simulation
    p.stepSimulation()

    # Get the new agent position from PyBullet
    agent_pos, _ = p.getBasePositionAndOrientation(agent_id)

    view = cam.get_view_matrix()
    glUniformMatrix4fv(view_loc, 1, GL_FALSE, view)

    # draw the maze
    glBindVertexArray(VAO[0])
    glBindTexture(GL_TEXTURE_2D, textures[0])
    cube_pos = pyrr.matrix44.create_from_translation(pyrr.Vector3([0, -3, -10]))
    glUniformMatrix4fv(model_loc, 1, GL_FALSE, cube_pos)
    glDrawArrays(GL_TRIANGLES, 0, len(cube_indices))
    
    # draw the agent
    glBindVertexArray(VAO[1])
    glBindTexture(GL_TEXTURE_2D, textures[1])
    model_matrix = pyrr.matrix44.create_from_translation(agent_pos)
    glUniformMatrix4fv(model_loc, 1, GL_FALSE, model_matrix)
    glDrawArrays(GL_TRIANGLES, 0, len(agent_indices))
    
    # draw the ground
    glBindVertexArray(VAO[2])
    glBindTexture(GL_TEXTURE_2D, textures[2])
    ground_pos = pyrr.matrix44.create_from_translation(pyrr.Vector3([0, -3, -10]))
    glUniformMatrix4fv(model_loc, 1, GL_FALSE, ground_pos)
    glDrawArrays(GL_TRIANGLES, 0, len(ground_indices))

    glfw.swap_buffers(window)
    
# Disconnect from PyBullet
p.disconnect()

# terminate glfw, free up allocated resources
glfw.terminate()

In [ ]:
p.configureDebugVisualizer(p.COV_ENABLE_RENDERING, 1)
p.stopStateLogging(logId)
p.setRealTimeSimulation(1)


colors = [[1, 0, 0, 1], [0, 1, 0, 1], [0, 0, 1, 1], [1, 1, 1, 1]]
currentColor = 0

while (1):
  time.sleep(1./240.)

In [3]:
#NO PYBULLET:

import glfw
from OpenGL.GL import *
from OpenGL.GL.shaders import compileProgram, compileShader
import pyrr
from TextureLoader import load_texture
from ObjLoader import ObjLoader
from camera import Camera
import math

cam = Camera()
WIDTH, HEIGHT = 1280, 720
lastX, lastY = WIDTH / 2, HEIGHT / 2
first_mouse = True
left, right, forward, backward = False, False, False, False


agent_pos = pyrr.Vector3([0.0, 3, 0.0])
agent_speed = 0.015


# the keyboard input callback
def key_input_clb(window, key, scancode, action, mode):
    global left, right, forward, backward
    if key == glfw.KEY_ESCAPE and action == glfw.PRESS:
        glfw.set_window_should_close(window, True)

    if key == glfw.KEY_W and action == glfw.PRESS:
        forward = True
    elif key == glfw.KEY_W and action == glfw.RELEASE:
        forward = False
    if key == glfw.KEY_S and action == glfw.PRESS:
        backward = True
    elif key == glfw.KEY_S and action == glfw.RELEASE:
        backward = False
    if key == glfw.KEY_A and action == glfw.PRESS:
        left = True
    elif key == glfw.KEY_A and action == glfw.RELEASE:
        left = False
    if key == glfw.KEY_D and action == glfw.PRESS:
        right = True
    elif key == glfw.KEY_D and action == glfw.RELEASE:
        right = False
    # if key in [glfw.KEY_W, glfw.KEY_S, glfw.KEY_D, glfw.KEY_A] and action == glfw.RELEASE:
    #     left, right, forward, backward = False, False, False, False


# do the movement, call this function in the main loop
def do_movement():
    if left:
        cam.process_keyboard("LEFT", 0.015)
    if right:
        cam.process_keyboard("RIGHT", 0.015)
    if forward:
        cam.process_keyboard("FORWARD", 0.0015)
    if backward:
        cam.process_keyboard("BACKWARD", 0.015)


# the mouse position callback function
def mouse_look_clb(window, xpos, ypos):
    global first_mouse, lastX, lastY

    if first_mouse:
        lastX = xpos
        lastY = ypos
        first_mouse = False

    xoffset = xpos - lastX
    yoffset = lastY - ypos

    lastX = xpos
    lastY = ypos

    cam.process_mouse_movement(xoffset, 0)


vertex_src = """
# version 330

layout(location = 0) in vec3 a_position;
layout(location = 1) in vec2 a_texture;
layout(location = 2) in vec3 a_normal;

uniform mat4 model;
uniform mat4 projection;
uniform mat4 view;

out vec2 v_texture;

void main()
{
    gl_Position = projection * view * model * vec4(a_position, 1.0);
    v_texture = a_texture;
}
"""

fragment_src = """
# version 330

in vec2 v_texture;

out vec4 out_color;

uniform sampler2D s_texture;

void main()
{
    out_color = texture(s_texture, v_texture);
}
"""


# the window resize callback function
def window_resize_clb(window, width, height):
    glViewport(0, 0, width, height)
    projection = pyrr.matrix44.create_perspective_projection_matrix(45, width / height, 0.1, 100)
    glUniformMatrix4fv(proj_loc, 1, GL_FALSE, projection)


# initializing glfw library
if not glfw.init():
    raise Exception("glfw can not be initialized!")

# creating the window
window = glfw.create_window(WIDTH, HEIGHT, "My OpenGL window", None, None)

# check if window was created
if not window:
    glfw.terminate()
    raise Exception("glfw window can not be created!")

# set window's position
glfw.set_window_pos(window, 400, 200)

# set the callback function for window resize
glfw.set_window_size_callback(window, window_resize_clb)
# set the mouse position callback
glfw.set_cursor_pos_callback(window, mouse_look_clb)
# set the keyboard input callback
glfw.set_key_callback(window, key_input_clb)
# capture the mouse cursor
glfw.set_input_mode(window, glfw.CURSOR, glfw.CURSOR_DISABLED)

# make the context current
glfw.make_context_current(window)

# load here the 3d meshes
cube_indices, cube_buffer = ObjLoader.load_model("models/mazeY_collider.obj")
agent_indices, agent_buffer = ObjLoader.load_model("models/chibi.obj")
ground_indices, ground_buffer = ObjLoader.load_model("models/groundY.obj")

shader = compileProgram(compileShader(vertex_src, GL_VERTEX_SHADER), compileShader(fragment_src, GL_FRAGMENT_SHADER))

# VAO and VBO
VAO = glGenVertexArrays(3)
VBO = glGenBuffers(3)

# cube VAO
glBindVertexArray(VAO[0])
# cube Vertex Buffer Object
glBindBuffer(GL_ARRAY_BUFFER, VBO[0])
glBufferData(GL_ARRAY_BUFFER, cube_buffer.nbytes, cube_buffer, GL_STATIC_DRAW)

# cube vertices
glEnableVertexAttribArray(0)
glVertexAttribPointer(0, 3, GL_FLOAT, GL_FALSE, cube_buffer.itemsize * 8, ctypes.c_void_p(0))
# cube textures
glEnableVertexAttribArray(1)
glVertexAttribPointer(1, 2, GL_FLOAT, GL_FALSE, cube_buffer.itemsize * 8, ctypes.c_void_p(12))
# cube normals
glVertexAttribPointer(2, 3, GL_FLOAT, GL_FALSE, cube_buffer.itemsize * 8, ctypes.c_void_p(20))
glEnableVertexAttribArray(2)


# Agent VAO
glBindVertexArray(VAO[1])
# Agent Vertex Buffer Object
glBindBuffer(GL_ARRAY_BUFFER, VBO[1])
glBufferData(GL_ARRAY_BUFFER, agent_buffer.nbytes, agent_buffer, GL_STATIC_DRAW)

# agent vertices
glEnableVertexAttribArray(0)
glVertexAttribPointer(0, 3, GL_FLOAT, GL_FALSE, agent_buffer.itemsize * 8, ctypes.c_void_p(0))
# agent textures
glEnableVertexAttribArray(1)
glVertexAttribPointer(1, 2, GL_FLOAT, GL_FALSE, agent_buffer.itemsize * 8, ctypes.c_void_p(12))
# agent normals
glVertexAttribPointer(2, 3, GL_FLOAT, GL_FALSE, agent_buffer.itemsize * 8, ctypes.c_void_p(20))
glEnableVertexAttribArray(2)


# Ground VAO
glBindVertexArray(VAO[2])
# Ground Vertex Buffer Object
glBindBuffer(GL_ARRAY_BUFFER, VBO[2])
glBufferData(GL_ARRAY_BUFFER, ground_buffer.nbytes, ground_buffer, GL_STATIC_DRAW)

# Ground vertices
glEnableVertexAttribArray(0)
glVertexAttribPointer(0, 3, GL_FLOAT, GL_FALSE, ground_buffer.itemsize * 8, ctypes.c_void_p(0))
# Ground textures
glEnableVertexAttribArray(1)
glVertexAttribPointer(1, 2, GL_FLOAT, GL_FALSE, ground_buffer.itemsize * 8, ctypes.c_void_p(12))
# Ground normals
glVertexAttribPointer(2, 3, GL_FLOAT, GL_FALSE, ground_buffer.itemsize * 8, ctypes.c_void_p(20))
glEnableVertexAttribArray(2)



textures = glGenTextures(3)
load_texture("textures/maze.jpg", textures[0])
load_texture("textures/chibi.png", textures[1])
load_texture("textures/ground.jpg", textures[2])

glUseProgram(shader)
glClearColor(0, 0.1, 0.1, 1)
glEnable(GL_DEPTH_TEST)
glEnable(GL_BLEND)
glBlendFunc(GL_SRC_ALPHA, GL_ONE_MINUS_SRC_ALPHA)

projection = pyrr.matrix44.create_perspective_projection_matrix(45, WIDTH / HEIGHT, 0.1, 100)


# eye, target, up
#view = pyrr.matrix44.create_look_at(pyrr.Vector3([0, 2, 16]), pyrr.Vector3([0, 0, 0]), pyrr.Vector3([0, 1, 0]))

model_loc = glGetUniformLocation(shader, "model")
proj_loc = glGetUniformLocation(shader, "projection")
view_loc = glGetUniformLocation(shader, "view")

glUniformMatrix4fv(proj_loc, 1, GL_FALSE, projection)
# glUniformMatrix4fv(view_loc, 1, GL_FALSE, view)

# the main application loop
while not glfw.window_should_close(window):
    glfw.poll_events()
    do_movement()

    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    
    # Move the agent based on the keyboard input
    if left:
        agent_pos.x -= agent_speed
    if right:
        agent_pos.x += agent_speed
    if forward:
        agent_pos.z -= agent_speed
    if backward:
        agent_pos.z += agent_speed
        
    #view = cam.get_view_matrix()
    view = cam.get_view_matrix(agent_pos)
    glUniformMatrix4fv(view_loc, 1, GL_FALSE, view)


    # draw the maze
    glBindVertexArray(VAO[0])
    glBindTexture(GL_TEXTURE_2D, textures[0])
    cube_pos = pyrr.matrix44.create_from_translation(pyrr.Vector3([0, 3, 0]))
    glUniformMatrix4fv(model_loc, 1, GL_FALSE, cube_pos)
    glDrawArrays(GL_TRIANGLES, 0, len(cube_indices))
    
    # draw the agent
    glBindVertexArray(VAO[1])
    glBindTexture(GL_TEXTURE_2D, textures[1])
    model_matrix = pyrr.matrix44.create_from_translation(agent_pos)
    glUniformMatrix4fv(model_loc, 1, GL_FALSE, model_matrix)
    glDrawArrays(GL_TRIANGLES, 0, len(agent_indices))
        
    # draw the ground
    glBindVertexArray(VAO[2])
    glBindTexture(GL_TEXTURE_2D, textures[2])
    ground_pos = pyrr.matrix44.create_from_translation(pyrr.Vector3([0, 3, 0]))
    glUniformMatrix4fv(model_loc, 1, GL_FALSE, ground_pos)
    glDrawArrays(GL_TRIANGLES, 0, len(ground_indices))

    glfw.swap_buffers(window)
    

# terminate glfw, free up allocated resources
glfw.terminate()

ValueError: invalid literal for int() with base 10: ''